In [1]:
import logging
from pathlib import Path

import geopandas as gpd
import matplotlib.pyplot as plt
import momepy
import networkx as nx
import numpy as np
import pandas as pd
import xarray as xr
from dotenv import dotenv_values

from generator_drainage_units import GeneratorCulvertLocations
from generator_drainage_units.utils.create_graph import create_graph_from_edges

logging.basicConfig(level=logging.DEBUG)

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
config = dotenv_values("..\\.env")

base_dir = Path(config["BASE_DIR"])
dir_basis_data = "0_basisdata"
dir_inter_results = "1_tussenresultaat"
dir_results = "2_resultaat"

# case_name = "vallei_en_veluwe"
case_name = "Leuvenumse_beek"

# base_dir = Path(config["BASE_DIR_AA_EN_MAAS"])
# dir_basis_data = "0_basisdata"
# dir_inter_results = None
# dir_results = "1_resultaat"

# case_name = "test"
# case_name = "test_zero"
# case_name = "test_check"
# case_name = "aa_en_maas"

case_path = Path(base_dir, case_name)

In [ ]:
cg = GeneratorCulvertLocations(
    path=case_path,
    read_results=True
)

In [22]:
nodes, edges, G = create_graph_from_edges(cg.overige_watergangen_processed, directed=False)

In [23]:
outflow_nodes = cg.outflow_points_overig_to_hydro.sjoin(nodes, how="inner").reset_index(drop=True)
outflow_nodes["coords"] = outflow_nodes["geometry"].apply(lambda x: x.coords[0])

In [ ]:
len_outflow_node, matrix = nx.multi_source_dijkstra(
    G, 
    [int(n) for n in outflow_nodes.nodeID.values], 
    weight="geom_length"
)
node_to_outflow_node = pd.DataFrame({
    "nodeID": matrix.keys(),
    "outflow_node": [v[0] for v in matrix.values()],
    "len_outflow_node": len_outflow_node.values()
})

In [44]:
nodes_connections = nodes.merge(
    node_to_outflow_node,
    how="outer",
    left_on="nodeID",
    right_on="nodeID"
)
nodes_connections["outflow_node"] = nodes_connections["outflow_node"].fillna(-999).astype(int)

In [ ]:
nodes_connections.to_file(Path(cg.path, cg.dir_inter_results, "nodes_connections.gpkg"))